<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1>Peer Review Final Assignment</h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
## get the data
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [2]:
#!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
#pip install --upgrade pip

Requirement already up-to-date: pip in /home/robi/.virtualenvs/DataScience/lib/python3.6/site-packages (20.1.1)
Note: you may need to restart the kernel to use updated packages.


In [1]:
#!pip uninstall tensorflow-tensorboard

#!pip install --upgrade numpy
#!pip install --upgrade keras
#!pip install --upgrade tensorflow
#!pip install tensorboard
#!pip install tf-nightly
#!pip install tensorflow==1.5.1


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator

import keras
from keras.models import Sequential
from keras.layers import Dense

#from keras.applications import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [3]:
print(keras.__version__)
print(tf.__version__)

2.4.3
2.2.0


In [4]:
num_classes = 2

image_resize = 224

batch_size_training = 80
batch_size_validation = 80

In [5]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [7]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [8]:
model = Sequential()

model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

model.add(Dense(num_classes, activation='softmax'))

print(model.layers)

model.layers[0].trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)

num_epochs = 2

[<tensorflow.python.keras.engine.training.Model object at 0x7f47527dc310>, <tensorflow.python.keras.layers.core.Dense object at 0x7f47536f4a30>]


In [13]:
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
376/376 [==============================] - 12290s 33s/step - loss: 0.0926 - accuracy: 0.9707 - val_loss: 0.0263 - val_accuracy: 0.9935
Epoch 2/2
376/376 [==============================] - 12989s 35s/step - loss: 0.0196 - accuracy: 0.9953 - val_loss: 0.0153 - val_accuracy: 0.9956


In [14]:
model.save('A4_classifier_VGG16_model.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [9]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical',
    shuffle=False)

Found 500 images belonging to 2 classes.


In [10]:
steps_per_epoch_test = len(test_generator)

In [11]:
RESNETmodel = keras.models.load_model('A3 classifier_resnet_model.h5')
VGG16model = keras.models.load_model('A4_classifier_VGG16_model.h5')

In [12]:
scoreRESNET = RESNETmodel.evaluate_generator(test_generator, steps_per_epoch_test)
print("Accuracy = ",scoreRESNET[1])

Instructions for updating:
Please use Model.evaluate, which supports generators.
Accuracy =  1.0


In [13]:
scoreVGG16 = VGG16model.evaluate_generator(test_generator, steps_per_epoch_test)
print("Accuracy = ",scoreVGG16[1])

Accuracy =  0.9959999918937683


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [14]:
predictRESNET = RESNETmodel.predict_generator(test_generator, steps_per_epoch_test)

Instructions for updating:
Please use Model.predict, which supports generators.


In [15]:
filenames = test_generator.filenames
cl = np.round(predictRESNET)
results = pd.DataFrame({"file":filenames,"pr":predictRESNET[:,0], "class":cl[:,0]})
results

,file,pr,class
0,negative/19751.jpg,0.999393,1.0
1,negative/19752.jpg,0.993904,1.0
2,negative/19753.jpg,0.975655,1.0
3,negative/19754.jpg,0.994978,1.0
4,negative/19755.jpg,0.999352,1.0
...,...,...,...
495,positive/19996.jpg,0.000063,0.0
496,positive/19997.jpg,0.000895,0.0
497,positive/19998.jpg,0.000021,0.0
498,positive/19999.jpg,0.000166,0.0


In [16]:
predictVGG16 = VGG16model.predict_generator(test_generator, steps_per_epoch_test)

In [17]:
cl = np.round(predictVGG16)
results = pd.DataFrame({"file":filenames,"pr":predictRESNET[:,0], "class":cl[:,0]})
results

,file,pr,class
0,negative/19751.jpg,0.999393,1.0
1,negative/19752.jpg,0.993904,1.0
2,negative/19753.jpg,0.975655,1.0
3,negative/19754.jpg,0.994978,1.0
4,negative/19755.jpg,0.999352,1.0
...,...,...,...
495,positive/19996.jpg,0.000063,0.0
496,positive/19997.jpg,0.000895,0.0
497,positive/19998.jpg,0.000021,0.0
498,positive/19999.jpg,0.000166,0.0


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).